# Sentiment Analysis

In [4]:
from transformers import pipeline

In [ ]:
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

Device set to use cpu


In [ ]:
import pandas as pd
from pathlib import Path

file_path = Path("data/processed-data")

csv_files = [f for f in file_path.glob("*.csv")]
dataframes = {f.stem: pd.read_csv(f) for f in csv_files}

In [ ]:
dataframes['guyguygyu']

In [ ]:
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)